# Module 2 Homework
In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the Colab covered at the livestream to re-use the code snippets.

## Imports python libraries

In [ ]:
# Import Python Libraries
import numpy as np
import pandas as pd
import requests
from IPython.display import display
from collections import Counter

# Finance data sources
import yfinance as yf
import pandas_datareader as pdr
# import eurostat
# import talib

# Data visualization libraries
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt

import time
from datetime import date
from io import StringIO

## Question 1: IPO Filings Web Scraping and Data Processing
What's the total sum ($m) of 2023 filings that happened on Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [ ]:
# Define headers with a user-agent to mimic a web browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# URL of the webpage
url = "https://stockanalysis.com/ipos/filings/"

response = requests.get(url, headers=headers)

ipos_filings_df = pd.read_html(StringIO(response.text))

In [ ]:
filings = ipos_filings_df[0]
filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     325 non-null    object
 1   Symbol          325 non-null    object
 2   Company Name    325 non-null    object
 3   Price Range     325 non-null    object
 4   Shares Offered  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [ ]:
filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
1,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
2,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
3,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
4,"Apr 22, 2024",DRJT,Derun Group Inc,$5.00,-


In [ ]:
# Create a function to calculate average price
def calculate_avg_price(price_range):
    if not price_range:
        return np.nan
    elif isinstance(price_range, float):
        # Convert float to string
        price_range = str(price_range)

    if '-' in price_range:
        min_price, max_price = map(float, price_range.split('-'))
        return (min_price + max_price) / 2
    else:
        return float(price_range)


# Create function to wrangle dataset
def wrangle(url):
    # Read data from url
    response = requests.get(url, headers=headers)
    df = pd.read_html(StringIO(response.text))
    df = df[0]

    # Convert Filing Date to datetime format
    df['Filing Date'] = pd.to_datetime(df['Filing Date'])

    # Convert  - in Price Range column to NAN and remove $ sign
    # df['Price Range'] = pd.to_numeric(df['Price Range'].str.replace('$', ''), errors='coerce')
    df['Price Range'] = df['Price Range'].str.replace('$', '').replace('-', float('nan'))

    # Convert Shares Offered column values to numeric
    df['Shares Offered'] = pd.to_numeric(df['Shares Offered'], errors='coerce')

    # Apply the calculate_avg_price function to create the 'Average Price' column
    df['Average Price'] = df['Price Range'].apply(calculate_avg_price)

    # Create Shares Offered Value column
    df['Shares_offered_value'] = df['Shares Offered'] * df['Average Price'] / 1e6

    # Create Day of Week and Year columns
    df['Day of Week'] = df['Filing Date'].dt.strftime('%A')
    df['Year'] = df['Filing Date'].dt.year
    # filings['Dayofweek'] = filings['Filing Date'].dt.dayofweek

    # Create a subset DataFrame for Year == 2023
    df = df[df['Year'] == 2023]

    # Create a subset DataFrame for Day of Week == Fridays
    df =  df[df['Day of Week'] == 'Friday']

    return df

In [ ]:
filings_2023 = wrangle(url)

In [ ]:
filings_2023.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Average Price,Shares_offered_value,Day of Week,Year
49,2023-12-29,LEC,Lafayette Energy Corp,3.50 - 4.50,1200000.0,4.0,4.8,Friday,2023
50,2023-12-29,EPSM,Epsium Enterprise Limited,NaN,NaN,NaN,NaN,Friday,2023
53,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,NaN,NaN,NaN,NaN,Friday,2023
54,2023-12-22,LZMH,LZ Technology Holdings Limited,NaN,NaN,NaN,NaN,Friday,2023
59,2023-12-15,GIT,Going International Holding Company Limited,NaN,NaN,NaN,NaN,Friday,2023


In [ ]:
filings_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 49 to 165
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           32 non-null     datetime64[ns]
 1   Symbol                32 non-null     object        
 2   Company Name          32 non-null     object        
 3   Price Range           25 non-null     object        
 4   Shares Offered        25 non-null     float64       
 5   Average Price         25 non-null     float64       
 6   Shares_offered_value  25 non-null     float64       
 7   Day of Week           32 non-null     object        
 8   Year                  32 non-null     int32         
dtypes: datetime64[ns](1), float64(3), int32(1), object(4)
memory usage: 2.4+ KB


In [ ]:
value = filings_2023['Shares_offered_value'].sum()
print(f'Filings during 2023, which happened on Fridays is ${value:.0f}m')

Filings during 2023, which happened on Fridays is $286m


## Question 2: IPOs "Fixed days hold" strategy
Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.
Additional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [ ]:
# Define a function to wrangle data from the endpoints and merge them
def merged_data(url1, url2, headers):

    # Read data from url1
    response1 = requests.get(url1, headers=headers)
    df1 = pd.read_html(StringIO(response1.text))
    df1 = df1[0]

    # Read data from url2
    response2 = requests.get(url2, headers=headers)
    df2 = pd.read_html(StringIO(response2.text))
    df2 = df2[0]

    # Merge the two DataFrames
    merged_df = pd.concat([df1, df2], ignore_index=True)

    # Convert Filing Date to datetime format
    merged_df['IPO Date'] = pd.to_datetime(merged_df['IPO Date'])

    # Convert IPO Price column
    merged_df['IPO Price'] = pd.to_numeric(merged_df['IPO Price'].str.replace('$', ''), errors='coerce')

    # Convert Current column
    merged_df['Current'] = pd.to_numeric(merged_df['Current'].str.replace('$', ''), errors='coerce')
    #  df['Price Range'] = df['Price Range'].str.replace('$', '').replace('-', float('nan'))

    # Convert Return column
    merged_df['Return'] = pd.to_numeric(merged_df['Return'].str.replace('%', ''), errors='coerce') / 100
    # merged_df['Return'] = merged_df['Return'].str.replace('%', '').replace('-', np.nan).astype(float) / 100

    # Create a subset of IPO Date < 2024-03-01
    merged_df =   merged_df[merged_df['IPO Date'] < '2024-03-01']

    # Remove the ticker 'RYZB'
    merged_df = merged_df[merged_df['Symbol'] != 'RYZB']

    return merged_df

In [ ]:
# Define headers with a user-agent to mimic a web browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

url1 = "https://stockanalysis.com/ipos/2023/"
url2 = "https://stockanalysis.com/ipos/2024/"

merged_df = merged_data(url1, url2, headers)

In [ ]:
merged_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,10.0,10.05,0.0050
1,2023-12-19,LGCB,Linkage Global Inc,4.0,3.10,-0.2250
2,2023-12-15,ZKH,ZKH Group Limited,15.5,12.34,-0.2039
3,2023-12-15,BAYA,Bayview Acquisition Corp,10.0,10.17,0.0170
4,2023-12-14,INHD,Inno Holdings Inc.,4.0,0.66,-0.8340


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      184 non-null    datetime64[ns]
 1   Symbol        184 non-null    object        
 2   Company Name  184 non-null    object        
 3   IPO Price     184 non-null    float64       
 4   Current       184 non-null    float64       
 5   Return        184 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 10.1+ KB


In [ ]:
stock_list = list(merged_df.Symbol.values)

In [ ]:
len(stock_list)

184

In [ ]:
stock_list.index('PTHR')

136

In [ ]:
stock_list[136]='PTHRU'

In [ ]:
end = date.today()
start = date(year=end.year-50, month=end.month, day=end.day)
print(f"period for indexes: {start} to {end}")

period for indexes: 1974-05-04 to 2024-05-04


In [ ]:
tickers = stock_list
df = yf.download(tickers, start=start, end=end, interval='1d')['Adj Close']

[*********************100%%**********************]  184 of 184 completed


In [ ]:
df.tail()

Ticker,AACT,AESI,AFJK,AHR,AITR,AIXI,ALCY,ANL,ANRO,ANSC,...,VHAI,VSME,VTMX,WBUY,WLGS,WRNT,YGFGF,YIBO,ZJYL,ZKH
Date,,,,,,,,,,,,,,,,,,,,,
2024-04-29,10.62,23.320000,10.275,13.87,10.31,1.30,10.64,14.80,15.56,10.235,...,0.1690,0.420,36.660000,0.3740,0.5350,0.270,0.002,2.10,3.77,11.90
2024-04-30,10.62,22.209999,10.260,13.73,10.31,1.22,10.64,13.80,15.51,10.235,...,0.1640,0.410,35.509998,0.3790,0.5470,0.300,0.002,2.08,3.81,11.50
2024-05-01,10.61,21.969999,10.275,13.88,10.31,1.22,10.64,14.13,16.10,10.225,...,0.1690,0.413,36.000000,0.3720,0.5350,0.324,0.002,2.06,3.63,11.96
2024-05-02,10.63,21.940001,10.275,13.97,10.31,1.25,10.70,13.02,14.40,10.235,...,0.1630,0.400,35.930000,0.3740,0.5800,0.318,0.002,2.10,3.52,12.05
2024-05-03,10.62,21.969999,NaN,13.73,10.32,1.20,10.67,13.78,14.22,10.235,...,0.1607,0.390,36.160000,0.4063,0.5802,0.298,NaN,2.05,3.62,12.34


In [ ]:
listing_dates = df.apply(lambda series: series.first_valid_index())

In [ ]:
listing_dates

Ticker
AACT    2023-06-13
AESI    2023-03-09
AFJK    2024-01-23
AHR     2024-02-07
AITR    2024-01-02
           ...    
WRNT    2023-07-25
YGFGF   2023-03-28
YIBO    2024-01-25
ZJYL    2023-03-28
ZKH     2023-12-15
Length: 184, dtype: datetime64[ns]

In [ ]:
value = listing_dates['ATGL']
value

Timestamp('2023-10-31 00:00:00')

In [108]:
def get_stock_growth(listing_dates, tickers, interval=30):
    """
    This function calculates the growth of stock prices for each stock from its listing date.

    Parameters:
    - listing_dates: A dictionary where keys are stock tickers and values are their listing dates.
    - tickers: A list of stock tickers.
    - interval: Number of days to calculate growth for (default is 30).

    Returns:
    - growth_df: A DataFrame containing the tickers as index, 'Min_Date' for the minimum date,
                 and 'Growth_1' to 'Growth_30' for the growth values over the 30-day interval.
    """

    # Create an empty DataFrame to store the results
    growth_df = pd.DataFrame(index=tickers)

    # Iterate through each stock ticker and its listing date
    for ticker, listing_date in listing_dates.items():
        # Download historical data from Yahoo Finance
        stock_data = yf.download(ticker, start=listing_date, end=pd.Timestamp.now().date(), progress=False)

        # Ensure that data is available for the specified interval
        if len(stock_data) >= interval + 1:  # Ensure at least interval + 1 days of data are available
            # Calculate the Min_Date for the stock
            min_date = stock_data.index.min()

            # Find the index of the minimum date and add one day to it
            start_index = stock_data.index.get_loc(min_date) + 1

            # Extract growth values over the 30-day interval starting from Min_Date
            growth_values = []
            for i in range(start_index, start_index + interval):
                future_price = stock_data['Adj Close'].shift(-i)
                growth = (future_price - stock_data['Adj Close']) / stock_data['Adj Close']
                # growth = stock_data['Adj Close'].shift(-i) / stock_data['Adj Close']
                growth_values.append(growth.iloc[start_index])  # Growth starting from the day after Min_Date

            # Store the growth values in the DataFrame
            growth_df.loc[ticker, 'Min_Date'] = min_date
            growth_df.loc[ticker, [f'Growth_future_{i}d' for i in range(1, interval + 1)]] = growth_values


    return growth_df

In [109]:
tickers = listing_dates.index.tolist()
growth_df = get_stock_growth(listing_dates, tickers)

In [110]:
growth_df.head()

,Min_Date,Growth_future_1d,Growth_future_2d,Growth_future_3d,Growth_future_4d,Growth_future_5d,Growth_future_6d,Growth_future_7d,Growth_future_8d,Growth_future_9d,...,Growth_future_21d,Growth_future_22d,Growth_future_23d,Growth_future_24d,Growth_future_25d,Growth_future_26d,Growth_future_27d,Growth_future_28d,Growth_future_29d,Growth_future_30d
AACT,2023-06-13,0.000990,0.000990,0.001980,0.001980,0.001980,0.003960,0.003960,0.004950,0.005941,...,0.005941,0.005941,0.005941,0.006931,0.006931,0.006931,0.008911,0.008911,0.008911,0.007921
AESI,2023-03-09,-0.030909,0.006667,-0.027273,-0.023030,-0.034545,-0.024242,-0.008485,-0.005454,-0.030303,...,0.078182,0.079394,0.087879,0.105455,0.104242,0.086667,0.095152,0.085454,0.101818,0.138788
AFJK,2024-01-23,0.000984,0.000984,0.001969,0.000984,0.001969,0.000984,0.000984,0.000492,0.000984,...,0.001476,0.001969,0.001378,0.001969,-0.000591,0.001476,0.002953,0.001969,0.002362,0.001969
AHR,2024-02-07,0.004594,0.006891,0.022205,0.053599,0.044410,0.062021,0.026799,0.026034,0.037519,...,0.060490,0.058193,0.053599,0.056662,0.040582,0.058959,0.064318,0.075038,0.085758,0.066616
AITR,2024-01-02,0.002956,0.002956,0.003054,0.003941,0.003941,0.003941,0.004434,0.003941,0.004926,...,0.005911,0.005911,0.006897,0.007882,0.006897,0.006700,0.006700,0.007389,0.007882,0.008867


In [ ]:
growth_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, AACT to ZKH
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Min_Date           182 non-null    datetime64[ns]
 1   Growth_future_1d   182 non-null    float64       
 2   Growth_future_2d   182 non-null    float64       
 3   Growth_future_3d   182 non-null    float64       
 4   Growth_future_4d   182 non-null    float64       
 5   Growth_future_5d   182 non-null    float64       
 6   Growth_future_6d   182 non-null    float64       
 7   Growth_future_7d   182 non-null    float64       
 8   Growth_future_8d   182 non-null    float64       
 9   Growth_future_9d   182 non-null    float64       
 10  Growth_future_10d  182 non-null    float64       
 11  Growth_future_11d  182 non-null    float64       
 12  Growth_future_12d  182 non-null    float64       
 13  Growth_future_13d  182 non-null    float64       
 14  Growth_futur

In [ ]:
growth_df[growth_df.Growth_future_1d.isnull()]

,Min_Date,Growth_future_1d,Growth_future_2d,Growth_future_3d,Growth_future_4d,Growth_future_5d,Growth_future_6d,Growth_future_7d,Growth_future_8d,Growth_future_9d,...,Growth_future_21d,Growth_future_22d,Growth_future_23d,Growth_future_24d,Growth_future_25d,Growth_future_26d,Growth_future_27d,Growth_future_28d,Growth_future_29d,Growth_future_30d
DYCQ,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LEGT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
growth_df.describe()

,Min_Date,Growth_future_1d,Growth_future_2d,Growth_future_3d,Growth_future_4d,Growth_future_5d,Growth_future_6d,Growth_future_7d,Growth_future_8d,Growth_future_9d,...,Growth_future_21d,Growth_future_22d,Growth_future_23d,Growth_future_24d,Growth_future_25d,Growth_future_26d,Growth_future_27d,Growth_future_28d,Growth_future_29d,Growth_future_30d
count,182,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,...,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000
mean,2023-07-11 10:17:08.571428608,-0.014584,-0.025620,-0.037883,-0.043556,-0.050087,-0.061733,-0.064694,-0.062755,-0.066730,...,-0.048454,-0.047297,-0.050201,-0.056540,-0.056980,-0.027980,-0.023337,-0.020373,-0.025585,-0.029997
min,2009-12-29 00:00:00,-0.331976,-0.440000,-0.480204,-0.711392,-0.673418,-0.718987,-0.769620,-0.764557,-0.736709,...,-0.823540,-0.836106,-0.837876,-0.841062,-0.856637,-0.895929,-0.887611,-0.875044,-0.876106,-0.880885
25%,2023-04-13 06:00:00,-0.031726,-0.079011,-0.118891,-0.140335,-0.170238,-0.181055,-0.209384,-0.209289,-0.206668,...,-0.312437,-0.323276,-0.312797,-0.326026,-0.337254,-0.351862,-0.341118,-0.317678,-0.325430,-0.313525
50%,2023-08-05 00:00:00,0.000000,-0.000395,-0.000996,-0.004357,-0.005578,-0.003185,-0.007002,-0.002327,-0.010746,...,-0.005586,-0.009143,-0.005660,-0.028902,-0.012365,-0.025437,-0.018693,-0.021685,-0.016955,-0.020339
75%,2023-11-16 06:00:00,0.019500,0.022566,0.021739,0.021394,0.013939,0.024738,0.018995,0.020457,0.015943,...,0.049028,0.041310,0.033088,0.022342,0.028868,0.032484,0.047881,0.038623,0.038440,0.015988
max,2024-03-11 00:00:00,0.623950,0.904000,1.088435,1.267574,1.164444,0.997733,1.018889,1.354445,1.666667,...,2.879819,2.854875,2.812018,2.554927,3.930116,8.056122,8.081632,8.265306,8.372449,8.290816
std,NaN,0.111297,0.165019,0.196727,0.235335,0.254601,0.241914,0.260586,0.288210,0.306354,...,0.461264,0.480336,0.486991,0.490962,0.531792,0.784622,0.785817,0.783343,0.769682,0.745978


In [121]:
# Compute 75% quantile growth for each holding period
quantile_75 = growth_df.groupby(growth_df.index)[[f'Growth_future_{i}d' for i in range(1, 31)]].quantile(0.75)

# Find the holding period (X) with the highest 75% quantile growth
optimal_X = quantile_75.mean(axis=0).idxmax()
print(optimal_X)

Growth_future_12d


In [ ]:
interval = 30
# Calculate the average growth for each day over the 30-day interval for each stock
average_growth_per_stock = growth_df[[f'Growth_future_{i}d' for i in range(1, interval + 1)]].mean(axis=1)

# # Identify the day with the highest average growth for each stock
# best_selling_days = average_growth_per_stock.groupby(level=0).idxmax()

# print("Best selling days for each stock based on average growth:")
# print(best_selling_days)

best_selling_days = {}

for ticker in average_growth_per_stock.index:
    max_value = None
    max_index = None

    # Iterate over the growth values and their corresponding index labels
    for index, value in growth_df.loc[ticker, [f'Growth_future_{i}d' for i in range(1, interval + 1)]].items():
        # Check if this value is the new maximum
        if max_value is None or value > max_value:
            max_value = value
            max_index = index

    # Store the result
    best_selling_days[ticker] = max_index

    # Count occurrences of each day in the result
day_counts = Counter(best_selling_days.values())

print("Count of each day in the result:")
print(day_counts)

print("Best selling days for each stock based on highest average growth:")
print(best_selling_days)

Count of each day in the result:
Counter({'Growth_future_1d': 42, 'Growth_future_30d': 16, 'Growth_future_2d': 16, 'Growth_future_29d': 10, 'Growth_future_28d': 7, 'Growth_future_27d': 6, 'Growth_future_22d': 6, 'Growth_future_14d': 5, 'Growth_future_25d': 5, 'Growth_future_12d': 5, 'Growth_future_26d': 5, 'Growth_future_20d': 5, 'Growth_future_4d': 5, 'Growth_future_16d': 4, 'Growth_future_24d': 4, 'Growth_future_13d': 4, 'Growth_future_3d': 4, 'Growth_future_5d': 4, 'Growth_future_15d': 3, 'Growth_future_18d': 3, 'Growth_future_10d': 3, 'Growth_future_11d': 3, 'Growth_future_9d': 3, 'Growth_future_7d': 3, 'Growth_future_21d': 3, 'Growth_future_23d': 3, 'Growth_future_19d': 2, 'Growth_future_8d': 2, 'Growth_future_6d': 2, 'Growth_future_17d': 1})
Best selling days for each stock based on highest average growth:
{'AACT': 'Growth_future_27d', 'AESI': 'Growth_future_30d', 'AFJK': 'Growth_future_15d', 'AHR': 'Growth_future_29d', 'AITR': 'Growth_future_30d', 'AIXI': 'Growth_future_16d', 'A

In [93]:
# Assuming 'growth_df' is your DataFrame with columns 'Growth_future_1d' to 'Growth_future_30d'
def calculate_average_growth(df, num_days):
    # Calculate the average growth over the rolling window
    average_growth = df.rolling(window=num_days, axis=1).mean()

    return average_growth

# Calculate average growth for each possible number of days
average_growth_values = []

for num_days in range(1, 31):
    average_growth = calculate_average_growth(growth_df, num_days)
    average_growth_values.append(average_growth)

# Create a DataFrame to store the average growth values
df_average_growth = pd.concat(average_growth_values, axis=1)

# Describe the DataFrame to get summary statistics
summary_stats = df_average_growth.describe()

# Extract the 75th percentile from the summary statistics
percentile_75 = summary_stats.loc['75%', :]

# Find the optimal number of days where the 75th percentile is highest
optimal_num_days = percentile_75.idxmax()

print("Optimal number of days with highest 75th percentile:", optimal_num_days)

Optimal number of days with highest 75th percentile: Growth_future_21d


## Question 3: Is Growth Concentrated in the Largest Stocks?
Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

Now let's add the top 12-22 stocks (as of end-April 2024):

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate growth_7d for every stock and every day. Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:

Date	ticker_category	growth_7d
2014-01-01	LARGE	1.011684
2014-01-01	LARGEST	1.011797
On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [ ]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS


NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

ALL_TICKERS = LARGEST_STOCKS + LARGE_STOCKS

In [ ]:
start = '2013-12-20'
end = '2023-12-31'
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                              start = start,
                              end = end,
                              interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns

  # historyPrices['growth_7d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(7)
  # historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  # historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  # historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  # historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  # historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  # historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  # historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

[*********************100%%**********************]  1 of 1 completed

0 MSFT


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


65 KOTAKBANK.NS


[*********************100%%**********************]  1 of 1 completed


66 TITAN.NS


[*********************100%%**********************]  1 of 1 completed


In [76]:
def get_ticker_type(ticker:str, largest_stocks_list, large_stocks_list):
  if ticker in largest_stocks_list:
    return 'LARGEST'
  elif ticker in large_stocks_list:
    return 'LARGE'
  else:
    return 'ERROR'

In [77]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, LARGEST_STOCKS, LARGE_STOCKS))

In [78]:
# count of observations between US-EU-INDIA stocks
stocks_df.ticker_type.value_counts()

ticker_type
LARGEST    83546
LARGE      81498
Name: count, dtype: int64

In [79]:
# unique tickers
stocks_df.Ticker.nunique()

65

In [80]:
stocks_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,LARGEST,2013-12-20,2023-12-29
ACN,LARGEST,2013-12-20,2023-12-29
ADANIENT.NS,LARGE,2013-12-20,2023-12-29
AI.PA,LARGE,2013-12-20,2023-12-29
AIR.PA,LARGE,2013-12-20,2023-12-29
...,...,...,...
TTE,LARGEST,2013-12-20,2023-12-29
UNH,LARGE,2013-12-20,2023-12-29
V,LARGEST,2013-12-20,2023-12-29


In [94]:
# Assuming you have a DataFrame 'stock_data' with columns: 'Date', 'Ticker', 'Close'
# Calculate growth_7d for each stock
stocks_df['growth_7d'] = stocks_df.groupby('Ticker')['Close'].pct_change(periods=7) + 1

In [99]:
# Create a pivot table
pivot_table = pd.pivot_table(stocks_df, values='growth_7d', index='Date', columns='ticker_type')

In [100]:
# Calculate average daily growth_7d for LARGE and LARGEST groups
average_growth_large = pivot_table['LARGE'].mean()
average_growth_largest = pivot_table['LARGEST'].mean()

# Calculate the percentage of days when LARGE outperforms LARGEST
total_days = len(pivot_table)
days_large_outperformed = len(pivot_table[pivot_table['LARGE'] > pivot_table['LARGEST']])
percentage_large_outperformed = int((days_large_outperformed / total_days) * 100)

print(f"Average daily growth_7d for LARGE group: {average_growth_large:.4f}")
print(f"Average daily growth_7d for LARGEST group: {average_growth_largest:.4f}")
print(f"Percentage of days when LARGE outperformed LARGEST: {percentage_large_outperformed}%")

Average daily growth_7d for LARGE group: 1.0024
Average daily growth_7d for LARGEST group: 1.0037
Percentage of days when LARGE outperformed LARGEST: 46%


## Question 4: Trying Another Technical Indicators strategy
What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this link.

Let's assume you've learned about the awesome CCI indicator (Commodity Channel Index), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (Date.dt.dayofweek()==4).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years? One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit. You need to sum these results over all trades (460 times in 10 years).

Additional:

Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
are you still profitable on those trades?

## [EXPLORATORY] Question 5: Finding Your Strategy for IPOs
You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?

Long-Term vs. Short-Term View:
Decide whether you’re looking for short-term gains (flipping) or long-term investment.
Some IPOs experience initial volatility but perform well over time.